In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.Connection("data/data2.sqlite")

In [14]:
q1 = """
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
)
;
"""
q1_result = pd.read_sql(q1, conn)
q1_result

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [16]:
q2 = """
SELECT p.productName, SUM(od.quantityOrdered) AS total_orders
FROM orderdetails od
JOIN products p ON od.productCode = p.productCode
GROUP BY p.productName
ORDER BY total_orders DESC
;
"""
q2_result = pd.read_sql(q2, conn)
q2_result

,productName,total_orders
0,1992 Ferrari 360 Spider red,1808
1,1937 Lincoln Berline,1111
2,American Airlines: MD-11S,1085
3,1941 Chevrolet Special Deluxe Cabriolet,1076
4,1930 Buick Marquette Phaeton,1074
...,...,...
104,1999 Indy 500 Monte Carlo SS,855
105,1911 Ford Town Car,832
106,1936 Mercedes Benz 500k Roadster,824
107,1970 Chevy Chevelle SS 454,803


In [18]:
q3 = """
SELECT p.productName, COUNT(DISTINCT o.customerNumber) AS total_customers
FROM orderdetails od
JOIN products p ON od.productCode = p.productCode
JOIN orders o ON od.orderNumber = o.orderNumber
GROUP BY p.productName
ORDER BY total_customers DESC
;
"""
q3_result = pd.read_sql(q3, conn)
q3_result

,productName,total_customers
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


In [20]:
q4 = """
SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, o.city, e.officeCode
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
WHERE e.employeeNumber IN (
    SELECT o.salesRepEmployeeNumber
    FROM orders o
    JOIN orderdetails od ON o.orderNumber = od.orderNumber
    WHERE od.productCode IN (
         SELECT od2.productCode
         FROM orderdetails od2
         JOIN orders o2 ON od2.orderNumber = o2.orderNumber
         GROUP BY od2.productCode
         HAVING COUNT(DISTINCT o2.customerNumber) < 20
    )
)
;
"""
q4_result = pd.read_sql(q1, conn)
q4_result

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [22]:
q5 = """
SELECT e.employeeNumber, e.firstName, e.lastName, c.num_customers
FROM employees e
JOIN (
    SELECT salesRepEmployeeNumber, COUNT(*) AS num_customers, AVG(creditLimit) AS avg_credit
    FROM customers
    GROUP BY salesRepEmployeeNumber
    HAVING AVG(creditLimit) > 15000
) c ON e.employeeNumber = c.salesRepEmployeeNumber
;
"""
q5_result = pd.read_sql(q5, conn)
q5_result

,employeeNumber,firstName,lastName,num_customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


In [27]:
conn.close()